## Distribution Centers

This notebook describes the preprocessing of data about pharmaceutical distribution centers in Germany.
The information is published by PHAGRO, the German Federal Association of Pharmaceutical Wholesalers.

The goal is to extract a list of all distribution centers, geocode them based on their addresses and store the data in the main pharmalink module.

### Source:

List of PHAGRO member's distribution centers, including addresses

[Website: scroll down to "Download Mitgliederliste"](https://www.phagro.de/mitglieder/) and [File](https://www.phagro.de/app/uploads/2024/01/PHAGRO-Jan-2024-Uebersicht-Mitglieder.pdf)

In [1]:
import pathlib as path
import requests as req
import pdfplumber
import pandas as pd
from shapely.geometry import Point
import time
import geopandas as gpd
from io import BytesIO
import lzma

In [2]:
# Establish notebook path for handling relative paths in the notebook
notebook_path = path.Path().resolve()

if notebook_path.stem != "distribution_centers":
    raise Exception(
        "Notebook file root must be set to parent directory of the notebook. Please resolve and re-run."
    )

In [3]:
# Ensure the file has been downloaded
pdf_path = notebook_path.joinpath("PHAGRO-Jan-2024-Uebersicht-Mitglieder.pdf")

if not pdf_path.exists():
    file_url = "https://www.phagro.de/app/uploads/2024/01/PHAGRO-Jan-2024-Uebersicht-Mitglieder.pdf"
    response = req.get(file_url)

    with open(pdf_path, "wb") as file:
        file.write(response.content)

In [4]:
# Open the pdf file with pdfplumber
with pdfplumber.open(pdf_path) as pdf:
    # Extract the tables from the second to fourth pages

    content = []
    for page in pdf.pages[1:4]:
        table = page.extract_table()

        # Remove the first row as it contains the column names
        table = table[1:]

        content.extend(table)

    # Extract the first row from the first page to get the column names
    columns = pdf.pages[1].extract_table()[0]

# Create a dataframe from the extracted content
dist_centers = pd.DataFrame(content, columns=columns)

In [5]:
# Clean the dataframe

# Keep only the relevant columns
dist_centers = dist_centers[["Firma", "Zusatz Firma", "Straße", "PLZ/Ort Straße"]]

# Rename the columns
dist_centers.columns = ["name", "ZUSATZ FIRMA", "street", "PLZ/ORT"]

# Transform empty strings and "None" to NaN
dist_centers = dist_centers.replace("", pd.NA).replace("None", pd.NA)

# Drop rows with no value in "Straße" and "PLZ/Ort"
dist_centers = dist_centers.dropna(subset=["street", "PLZ/ORT"])

# Reset the index
dist_centers = dist_centers.reset_index(drop=True)

# Split the "PLZ/Ort" column into two columns "PLZ" and "Ort"
# Extract the first 5 numbers as the postal code and the rest as the city
dist_centers["PLZ"] = dist_centers["PLZ/ORT"].str.extract(r"(\d{5})")
dist_centers["city"] = dist_centers["PLZ/ORT"].str.extract(r"\d{5}\s(.*)")
dist_centers = dist_centers.drop(columns=["PLZ/ORT"])

# Split the "Straße" column into two columns "Straße" and "Hausnummer"
# Extract numbers, dashes and pluses after the last space as the house number and the rest as the street name
dist_centers["number"] = dist_centers["street"].str.extract(r"\s([\d\+\-]+)$")
dist_centers["street"] = dist_centers["street"].str.extract(r"^(.*)\s[\d\+\-]+$")

# Combine the "name" and "ZUSATZ FIRMA" columns into one column "name"
dist_centers["name"] = dist_centers["name"].str.cat(
    dist_centers["ZUSATZ FIRMA"], sep=" ", na_rep=""
)
dist_centers = dist_centers.drop(columns=["ZUSATZ FIRMA"])

# Reorder the columns
dist_centers = dist_centers[["name", "street", "number", "PLZ", "city"]]

# Replace \n with a space in the "Firma" and Zusatz Firma columns
dist_centers["name"] = dist_centers["name"].str.replace("\n", " ")

# Remove all urls from the "Firma" and "Zusatz Firma" columns
# The urls are in the format "www.example.de/com/eu"
pattern = r"www\.[a-zA-Z0-9-]+\.(de|com|eu)"
dist_centers["name"] = dist_centers["name"].str.replace(pattern, "", regex=True)

In [6]:
dist_centers

,name,street,number,PLZ,city
0,Alliance Healthcare Deutschland GmbH Zentrale,Solmsstraße,73,60486,Frankfurt / Main
1,Alliance Healthcare Deutschland GmbH Stuttgart,Pragstraße,154,70376,Stuttgart
2,Alliance Healthcare Deutschland GmbH Niederlas...,Anzag,1,87787,Wolfertschwenden/Allgäu
3,Alliance Healthcare Deutschland GmbH Niederlas...,Neues Ufer,13,10553,Berlin
4,Alliance Healthcare Deutschland GmbH Niederlas...,Ingolstädter Straße,5,28219,Bremen
...,...,...,...,...,...
104,Sanacorp eG Pharmazeutische Großhandlung Niede...,Theodor-Heuss-Straße,9,66130,Saarbrücken
105,Sanacorp eG Pharmazeutische Großhandlung Niede...,Nesebanzer Weg,3,18439,Stralsund
106,Sanacorp eG Pharmazeutische Großhandlung Niede...,Alemannenstraße,10,78532,Tuttlingen
107,Sanacorp eG Pharmazeutische Großhandlung Niede...,Hans-Lorenser-Straße,30,89079,Ulm


In [7]:
# Geocode the distribution centers using the Nominatim API.
# The Nominatim API is a search engine for OpenStreetMap data.
# This should take around 2 minutes to complete, as we are waiting 1 second between each request to avoid rate limiting.
nominatim_url = "https://nominatim.openstreetmap.org/search"
with req.Session() as session:
    nominatim_url = "https://nominatim.openstreetmap.org/search"

    # Update the session headers to comply with the Nominatim usage policy
    headers = {"user-agent": "pharmalink"}
    session.headers.update(headers)

    # Iterate over the rows and get the coordinates for each distribution center
    for index, row in dist_centers.iterrows():
        params = {
            "format": "json",
            "street": f"{row["street"]} {row["number"]}",
            # "city": row["city"], # This is quite unreliable, as the city name can differ from the one in OSM
            "postalcode": row["PLZ"],
            "country": "Germany",
            "addressdetails": 1,
        }

        response = session.get(nominatim_url, params=params)
        response.raise_for_status()

        data = response.json()

        if data:
            location = Point(float(data[0]["lon"]), float(data[0]["lat"]))
            dist_centers.loc[index, "location"] = location
        else:
            dist_centers.loc[index, "location"] = pd.NA
            print(
                f"Could not find coordinates for {row["name"]} at {row["street"]} {row["number"]}, {row["PLZ"]} {row["city"]}"
            )

        # Wait for 1 second to avoid rate limiting
        time.sleep(1)

Could not find coordinates for Alliance Healthcare Deutschland GmbH Niederlassung Allgäu at Anzag 1, 87787 Wolfertschwenden/Allgäu


In [8]:
# Manual correction is necessary for one of the distribution centers
# Due to some mysterious error in the OSM data (that escapes me, i've tried to resolve it on OSM),
# we need to add coordinates for the Alliance Healthcare Deutschland AG distribution center in 87787 Wolfertschwenden

# OSM way id for the building:
way_id = 289308460

# Use the OSM API to get the coordinates of the building
overpass_url = "https://overpass-api.de/api/interpreter"
overpass_query = f"""
    [out:json];
    way({way_id});
    out center;
"""

response = req.post(overpass_url, data=overpass_query)
response.raise_for_status()

data = response.json()

if data["elements"]:
    location = Point(
        data["elements"][0]["center"]["lon"], data["elements"][0]["center"]["lat"]
    )

# Update the coordinates for the Alliance Healthcare Deutschland AG distribution center
dist_centers.at[2, "location"] = location

In [9]:
# Transform the dataframe to a GeoDataFrame
dist_centers = gpd.GeoDataFrame(dist_centers, geometry="location", crs="EPSG:4326")

In [10]:
output_file = notebook_path.joinpath("distribution_centers.gpkg.xz")

# Delete the output file if it already exists
if output_file.exists():
    output_file.unlink()


# Write the GeoDataFrame to a compressed GeoPackage file using a BytesIO buffer
with BytesIO() as buffer:
    dist_centers.to_file(buffer, layer="distribution_centers", driver="GPKG")
    buffer.seek(0)
    with lzma.open(output_file, "wb", preset=9) as archive:
        archive.write(buffer.read())